# scratch

In [1]:
from brainlit.utils.write import czi_to_zarr, zarr_to_omezarr, _read_czi_slice
import zarr
from cloudvolume import CloudVolume
import json
from pathlib import Path
import aicspylibczi
import numpy as np

In [2]:
project_path = "/Users/thomasathey/Documents/mimlab/mouselight/brainlit_parent/brainlit/experiments/sriram/data/"  # "C:\\Users\\Sriram Sudarsanam\\Desktop\\NeuroglancerTrial\\"
czi_path = f"{project_path}test.czi"  # path to czi image
out_dir = f"{project_path}"  # path to directory where zarr should be made, should end in slash

In [6]:
czi = aicspylibczi.CziFile(czi_path)
slice1 = _read_czi_slice(czi, C=0, Z=0)
C = czi.get_dims_shape()[0]["C"][1]
H = slice1.shape[0]
W = slice1.shape[1]
Z = czi.get_dims_shape()[0]["Z"][1]

print(
    f"Writing {C} zarrs of shape {H}x{W}x{Z} from czi with dims {czi.get_dims_shape()}"
)
sz = np.array([H, W, Z], dtype="int")

fg_path = out_dir + "fg.zarr"
zarr_fg = zarr.open(
    fg_path, mode="w", shape=sz, chunks=(200, 200, 10), dtype="uint16"
)

Writing 2 zarrs of shape 1998x2009x40 from czi with dims [{'X': (0, 2009), 'Y': (0, 1998), 'Z': (0, 40), 'C': (0, 2), 'T': (0, 1), 'V': (0, 1), 'B': (0, 1)}]


In [7]:
def _write_zrange_thread(zarr_path, czi_path, channel, zs):
    czi = aicspylibczi.CziFile(czi_path)

    zarr_fg = zarr.open(
        zarr_path
    )
    for z in zs:
        zarr_fg[:, :, z] = _read_czi_slice(czi, C=channel, Z=z)

In [8]:
from joblib import Parallel, delayed

z_blocks = [np.arange(i,i+10) for i in [0,10,20,30]]

Parallel(n_jobs=4)(delayed(_write_zrange_thread)(fg_path, czi_path, 1, zs) for zs in z_blocks)

[None, None, None, None]

In [21]:
import json

fname = "/Users/thomasathey/Documents/mimlab/mouselight/brainlit_parent/brainlit/experiments/sriram/traces/skeletons/info"
with open(fname) as f:
    data = json.load(f)
    print(data["vertex_attributes"])
    for i, attr in enumerate(data["vertex_attributes"]):
        if attr["id"] == "vertex_types":
            data["vertex_attributes"].pop(i)
            break

with open(fname, "w") as f:
    json.dump(data, f)

[{'id': 'radius', 'data_type': 'float32', 'num_components': 1}, {'id': 'vertex_types', 'data_type': 'uint8', 'num_components': 1}]


In [18]:
store = parse_url(
    "/Users/thomasathey/Documents/jovolab/neurodata_infrastructure_tips/neuroglancer-zarr/test_ngff.zarr",
    mode="w",
).store
root = zarr.group(store=store)
write_image(image=dra, group=root, axes="xyz")

python cors_webserver.py -d "/Users/thomasathey/Documents/jovolab/neurodata_infrastructure_tips/neuroglancer-zarr/" -p 9010

In [35]:
ng_path = "precomputed://file:///Users/thomasathey/Documents/mimlab/mouselight/brainlit_parent/brainlit/experiments/sriram/sample/ng/frags"

vol_im = CloudVolume(ng_path)
vol_im.chunk_size
vol_im.resolution

Vec(500,500,300, dtype=int64)

In [36]:
ng_path = "precomputed://file:///Users/thomasathey/Documents/mimlab/mouselight/brainlit_parent/brainlit/experiments/sriram/sample/ng/traces"

info = CloudVolume.create_new_info(
    num_channels=1,
    layer_type="segmentation",
    data_type="uint16",  # Channel images might be 'uint8'
    # raw, png, jpeg, compressed_segmentation, fpzip, kempressed, zfpc, compresso
    encoding="raw",
    resolution=vol_im.resolution,  # Voxel scaling, units are in nanometers
    voxel_offset=[0, 0, 0],  # x,y,z offset in voxels from the origin
    # Pick a convenient size for your underlying chunk representation
    # Powers of two are recommended, doesn't need to cover image exactly
    chunk_size=vol_im.chunk_size,  # units are voxels
    volume_size=vol_im.shape[:3],  # e.g. a cubic millimeter dataset
    skeletons="skeletons",
)
vol = CloudVolume(ng_path, info=info, compress=False)
vol.commit_info()

## Read ome-zarr

In [11]:
import ome_zarr
import ome_zarr.reader
import numpy as np

In [29]:
file_path = "/Users/thomasathey/Documents/jovolab/neurodata_infrastructure_tips/neuroglancer-zarr/test_ngff.zarr"
loc = ome_zarr.io.parse_url(file_path, mode="r")
zarr_reader = ome_zarr.reader.Reader(loc).zarr
res0 = zarr_reader.load("0")

In [30]:
full_image_npy = res0.compute()
print(full_image_npy.shape)

(100, 100, 100)


In [18]:
sub_image = res0[:, :, :]
print(sub_image)
sub_image_npy = sub_image.compute()
print(np.unique(sub_image_npy))

dask.array<from-zarr, shape=(100, 100, 100), dtype=int64, chunksize=(50, 50, 100), chunktype=numpy.ndarray>
[0 1]


## Write zarr

In [10]:
import numpy as np
import zarr
from ome_zarr.io import parse_url
from ome_zarr.writer import write_image

# Modify base_dir below
base_dir = "/Users/thomasathey/Documents/jovolab/neurodata_infrastructure_tips/ng-zarr-test/"  # <path-to-ng-zarr-data>

# Image is cube of zeros with cube of 1's in the middle
data = np.zeros((100, 100, 100))
data[40:60, 40:60, 40:60] = 1
data = data.astype("uint64")

# Write zarr
z = zarr.open(
    base_dir + "test.zarr",
    mode="w",
    shape=data.shape,
    chunks=data.shape,
    dtype="uint64",
)
z[:, :, :] = data

# Write ome-zarr
path = base_dir + "test-ome.zarr"
store = parse_url(path, mode="w").store
root = zarr.group(store=store)
write_image(image=data, group=root, axes="zyx", storage_options=dict(chunks=data.shape))

uint64


### In neuroglancer directory:

python cors_webserver.py -d "<base_dir>" -p 9010

### Add zarr data image layer:
zarr://http://127.0.0.1:9010/test.zarr/

### Add ome-zarr data image layer:
zarr://http://127.0.0.1:9010/test-ome.zarr/